In [ ]:
import pandas as pd
from itertools import combinations

# Define the Binary Tree Node class (iterative version)
class BinaryTreeNode:
    def __init__(self, key):
        self.key = key
        self.count = 1  # Initially, count is 1
        self.left = None
        self.right = None

class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, key):
        """Insert an itemset into the binary tree iteratively."""
        if self.root is None:
            self.root = BinaryTreeNode(key)
        else:
            current = self.root
            while True:
                if key == current.key:
                    current.count += 1
                    return
                elif key < current.key:
                    if current.left is None:
                        current.left = BinaryTreeNode(key)
                        return
                    current = current.left
                else:
                    if current.right is None:
                        current.right = BinaryTreeNode(key)
                        return
                    current = current.right

    def in_order(self):
        """Iteratively traverse the tree and return itemsets in sorted order."""
        stack = []
        current = self.root
        itemsets = []

        while stack or current:
            while current:
                stack.append(current)
                current = current.left
            current = stack.pop()
            itemsets.append((current.key, current.count))
            current = current.right

        return itemsets

# Apriori Algorithm
def apriori(transactions, min_support):
    # Step 1: Create single itemsets and calculate their support
    itemsets = BinaryTree()
    for transaction in transactions:
        for item in transaction:
            itemsets.insert(frozenset([item]))

    # Step 2: Filter itemsets by min_support
    frequent_itemsets = []
    itemsets_list = itemsets.in_order()
    for itemset, count in itemsets_list:
        support = count / len(transactions)
        if support >= min_support:
            frequent_itemsets.append((itemset, support))

    # Step 3: Generate candidate itemsets of size 2 and higher
    current_itemsets = frequent_itemsets
    k = 2  # Start with pairs of items
    while current_itemsets:
        next_itemsets = []
        # Generate candidate itemsets by combining previous frequent itemsets
        candidates = generate_candidates(current_itemsets, k)
        for candidate in candidates:
            count = sum(1 for transaction in transactions if candidate.issubset(transaction))
            support = count / len(transactions)
            if support >= min_support:
                next_itemsets.append((candidate, support))
        current_itemsets = next_itemsets
        frequent_itemsets.extend(next_itemsets)
        k += 1  # Increase the size of itemsets

    return frequent_itemsets

def generate_candidates(frequent_itemsets, k):
    """Generate candidate itemsets of size k by combining frequent itemsets."""
    candidates = []
    for i in range(len(frequent_itemsets)):
        for j in range(i + 1, len(frequent_itemsets)):
            itemset1, _ = frequent_itemsets[i]
            itemset2, _ = frequent_itemsets[j]
            union = itemset1.union(itemset2)
            if len(union) == k:  # Ensure it's a valid itemset of size k
                candidates.append(union)
    return candidates

# Load transactions from a .txt file
def load_transactions(file_path):
    transactions = []
    with open(file_path, 'r') as file:
        for line in file:
            items = set(line.strip().split())  # Split items by space or comma
            transactions.append(items)
    return transactions

# Main function
def main():
    # Read transactions from 'transactions.txt'
    transactions = load_transactions('transactions.txt')

    # Run Apriori algorithm with minimum support (e.g., 20%)
    min_support = 0.2
    frequent_itemsets = apriori(transactions, min_support)

    # Save the results to a CSV file
    frequent_itemsets_df = pd.DataFrame(frequent_itemsets, columns=['Itemset', 'Support'])
    frequent_itemsets_df['Itemset'] = frequent_itemsets_df['Itemset'].apply(lambda x: ', '.join(sorted(x)))
    frequent_itemsets_df.to_csv('freq_patterns.csv', index=False)

    print("Frequent itemsets saved to 'freq_patterns.csv'")

# Run the main function
if __name__ == '__main__':
    main()
